# Initializing 

### Import/Set up

In [1]:
import time 

start = time.time()

In [2]:
import matplotlib 
matplotlib.use('Agg')

In [3]:
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import numpy as np
import seaborn as sns 

%matplotlib inline
#%config Completer.use_jedi = False 

In [4]:
from brian2 import *
import brian2genn

# outputDirectory = '/work/kbw29/brian2Genn/Output'
outputDirectory = '/hpc/home/kbw29/Code/brian2Genn/Output'


set_device('genn', directory = outputDirectory)

INFO       The following preferences have been changed for Brian2GeNN, reset them manually if you use a different device later in the same script: codegen.loop_invariant_optimisations, core.network.default_schedule [brian2.devices.genn]


#### Personal Functions 

In [5]:
def visualise_connectivity(S):
    Ns = len(S.source)
    Nt = len(S.target)
    figure(figsize=(10, 4))
    subplot(121)
    plot(zeros(Ns), arange(Ns), 'ok', ms=10)
    plot(ones(Nt), arange(Nt), 'ok', ms=10)
    for i, j in zip(S.i, S.j):
        plot([0, 1], [i, j], '-k')
    xticks([0, 1], ['Source', 'Target'])
    ylabel('Neuron index')
    xlim(-0.1, 1.1)
    ylim(-1, max(Ns, Nt))
    subplot(122)
    plot(S.i, S.j, 'ok')
    xlim(-1, Ns)
    ylim(-1, Nt)
    xlabel('Source neuron index')
    ylabel('Target neuron index')

In [6]:
def plot_spikes(monitor, stateMonitor, rangeNeurons):
    for i in range(rangeNeurons):
        spikes = (monitor.t[monitor.i == i] - defaultclock.dt)/ms
        val = stateMonitor[i].V
        subplot(rangeNeurons, 1, i+1)
        plot(tile(spikes, (2, 1)), vstack((val[array(spikes, dtype=int)], zeros(len(spikes)))), 'C0')
        title("Spikes")
    tight_layout()
    show()

In [7]:
def plot_potential(monitor, stateMonitor, rangeNeurons):
    for i in range(rangeNeurons):
        spikes = (monitor.t[monitor.i == i] - defaultclock.dt)/ms
        val = stateMonitor[i].V
        subplot(rangeNeurons, 1, i+1)
        plot(stateMonitor.t/ms, val)
        title("Trace")
    tight_layout()
    show()

In [8]:
def plot_population(prMonitorRun, spikeMonitorRun, labelGroup, nActivity, pRun): 
    # plotting
    title('Population rates')
    xlabel('ms')
    ylabel('Hz')

    plot(prMonitorRun.t / ms, prMonitorRun.smooth_rate(width=25 * ms) / Hz, label=labelGroup)

    legend()
    figure()

    title('Population activities ({} neurons/pop)'.format(nActivity))
    xlabel('ms')
    yticks([])

    plot(spikeMonitorRun.t / ms, spikeMonitorRun.i + (pRun + 1) * nActivity, '.', markersize=2, label=labelGroup) 


    legend()
    show()

In [9]:
def plot_population_Multi(prMonitorRun_smooth, prMonitorRun_t, spikeMonitorRun, labelGroup, nActivity, pRun): 
    # plotting
    title('Population rates')
    xlabel('ms')
    ylabel('Hz')

    plot(prMonitorRun_t / ms, prMonitorRun_smooth, label=labelGroup)

    legend()
    figure()

    title('Population activities ({} neurons/pop)'.format(nActivity))
    xlabel('ms')
    yticks([])

    plot(spikeMonitorRun.t / ms, spikeMonitorRun.i + (pRun + 1) * nActivity, '.', markersize=2, label=labelGroup) 


    legend()
    show()

In [10]:
def plot_population_Multi_2(prMonitorRun_smooth, prMonitorRun_t, labelGroup): 
    # plotting
    title('Population rates')
    xlabel('ms')
    ylabel('Hz')

    plot(prMonitorRun_t / ms, prMonitorRun_smooth, label=labelGroup)

    legend()
    show()

In [11]:
def active_percent(monitor, binSize, durationSimulation, numNeurons, ymax):
    # Define bin size 
    bin_size = binSize*ms
    # Define duration of simulation 
    duration = durationSimulation*second
    # Define number of neurons 
    num_Neurons = numNeurons

    # Initialize array to record spike occurancces 
    spkTotal = np.zeros([int(duration/bin_size)])
    
    for current in range(num_Neurons): 
        spk_count, bin_edges = np.histogram(a = np.r_[monitor.t[monitor.i == current]/second],
                                            bins = int(duration/bin_size), range = (0,durationSimulation))
        # Convert spike count to binary 
        binary = np.where(spk_count > 0, 1, 0)
        # Add to recording array
        spkTotal = spkTotal + binary

    plt.plot(bin_edges[:-1], (spkTotal/num_Neurons)*100)
    plt.axis([0, durationSimulation, 0, ymax])
    plt.title('Percent active granule cells as function of time')
    plt.ylabel('Active Cell (%)')
    plt.xlabel('Time (s)')
    plt.show()

In [12]:
def averaged_ActivePercent(spkMulti, spkSingle, binSize, durationSimulation, numNeurons, ymax):
    # Define bin size 
    bin_size = binSize*ms
    # Define duration of simulation 
    duration = durationSimulation*second
    # Define number of neurons 
    num_Neurons = numNeurons
    
    spk_count, bin_edges = np.histogram(a = np.r_[spkSingle.t[spkSingle.i == 1]/second],
                                        bins = int(duration/bin_size), range = (0,durationSimulation))

    averaged_spkMulti = np.mean(spkMulti, axis = 0)
    
    plt.plot(bin_edges[:-1], averaged_spkMulti)
    plt.axis([0, durationSimulation, 0, ymax])
    plt.title('Percent active granule cells as function of time')
    plt.ylabel('Active Cell (%)')
    plt.xlabel('Time (s)')
    plt.show()

# Neuron Models

### Granule Cells 

In [13]:
# Parameters 

# neuron parameters
theta_GR = -35*mV # threshold (from Yamazaki) 
Cm_GR = 3.1*pF 

# Conductances 
gL_GR = 0.43*nS 
g_AMPA_GR = 0.18*nS 
g_NMDA_GR = 0.025*nS 
g_Inh_GR = 0.028*nS 
g_ahp_GR = 1*nS

# Various leaks (from Yamazaki) 
El_GR = -58*mV
Eexc = 0*mV 
E_inh_GR = -82*mV 
E_ahp_GR = -82*mV # after hyperpolarization 

# Tau
tau_ahp_GR = 5*ms 
tau_AMPA_GR = 1.2*ms 
tau_NMDA_GR = 52.0*ms 
tau_GABA_1_GR = 7*ms
tau_GABA_2_GR = 59*ms

eqs_GR = Equations('''
                        dV / dt = (-gL_GR * (V - El_GR) - I_syn) / Cm_GR : volt 
                        
                        I_syn = I_AMPA + I_NMDA + I_GABA_tot + I_ahp: amp 
                        I_GABA_tot = 0.43*I_GABA_1 + 0.57*I_GABA_2 : amp 
                        
                        I_AMPA = g_AMPA_GR * (V - Eexc) * s_AMPA : amp
                        ds_AMPA / dt = -s_AMPA / tau_AMPA_GR : 1 
                        
                        I_NMDA = g_NMDA_GR * (V - Eexc) * s_NMDA : amp
                        ds_NMDA / dt = -s_NMDA / tau_NMDA_GR : 1 
                        
                        I_GABA_1 = g_Inh_GR * (V - E_inh_GR) * s_GABA_1 : amp
                        ds_GABA_1 / dt = -s_GABA_1 / tau_GABA_1_GR : 1 
                        
                        I_GABA_2 = g_Inh_GR * (V - E_inh_GR) * s_GABA_2 : amp
                        ds_GABA_2 / dt = - s_GABA_2 / tau_GABA_2_GR : 1 
                        
                        I_ahp = g_ahp_GR * (V - E_ahp_GR) * s_ahp_GR: amp
                        ds_ahp_GR / dt = - s_ahp_GR / tau_ahp_GR : 1
                        
                        x : metre 
                        y : metre 
                ''')

# Defining granule cell neuron group 
granuleCells = NeuronGroup(N=10, model=eqs_GR, reset='V = El_GR',
                    threshold='V > theta_GR',
                    method='euler')
granuleCells.V = El_GR 

In [14]:
# Defining MF input 

# Defining input parameters 
num_inputs = 100
f = 0.5*Hz 
eq_rate = '(22.5 + 7.5*cos((2*pi*t*f) + pi))*Hz'

# Equations 
eqs_glut_MFGR = '''
                   w_MFGR : 1 
                '''
eqs_pre_glut_MFGR = '''
                       s_AMPA += 1
                    '''

P_MFGR = PoissonGroup(num_inputs, rates = eq_rate)
S_MFGR = Synapses(P_MFGR, granuleCells, model = eqs_glut_MFGR, on_pre = eqs_pre_glut_MFGR, method = 'euler') 
S_MFGR.connect(condition = 'i == j')
S_MFGR.w_MFGR = 4


In [15]:
# Visualize synaptic connections 
#visualise_connectivity(S_MFGR)

# Running simulation 

In [16]:
time.time() - start

2.995042562484741

In [17]:
# run the simulation
BrianLogger.log_level_info()
run(2*second)

running brian code generation ...
building genn executable ...
['/hpc/home/kbw29/Code/genn/bin/genn-buildmodel.sh', '-i', '/hpc/home/kbw29/Code/brian2Genn:/hpc/home/kbw29/Code/brian2Genn/Output:/hpc/home/kbw29/Code/brian2Genn/Output/brianlib/randomkit', 'magicnetwork_model.cpp']
make: Entering directory '/hpc/home/kbw29/Code/genn/src/genn/generator'
if [ -w /hpc/home/kbw29/Code/genn/lib ]; then make -C /hpc/home/kbw29/Code/genn/src/genn/genn; fi;
if [ -w /hpc/home/kbw29/Code/genn/lib ]; then make -C /hpc/home/kbw29/Code/genn/src/genn/backends/cuda; fi;
make[1]: Entering directory '/hpc/home/kbw29/Code/genn/src/genn/genn'
make[1]: Entering directory '/hpc/home/kbw29/Code/genn/src/genn/backends/cuda'
make[1]: Nothing to be done for 'all'.
make[1]: Leaving directory '/hpc/home/kbw29/Code/genn/src/genn/backends/cuda'
make[1]: Nothing to be done for 'all'.
make[1]: Leaving directory '/hpc/home/kbw29/Code/genn/src/genn/genn'
mkdir -p /hpc/home/kbw29/Code/brian2Genn/Output
g++ -std=c++11 -Wal

In file included from /hpc/home/kbw29/Code/brian2Genn/Output/magicnetwork_model.cpp:5:0,
                 from generator.cc:31:
/hpc/home/kbw29/Code/brian2Genn/Output/brianlib/randomkit/randomkit.cc:132:1: warning: ISO C++ forbids converting a string constant to ‘char*’ [-Wwrite-strings]
 };
 ^
/hpc/home/kbw29/Code/brian2Genn/Output/brianlib/randomkit/randomkit.cc:132:1: warning: ISO C++ forbids converting a string constant to ‘char*’ [-Wwrite-strings]
In file included from /hpc/home/kbw29/Code/brian2Genn/Output/objects.h:6:0,
                 from /hpc/home/kbw29/Code/brian2Genn/Output/magicnetwork_model.cpp:7,
                 from generator.cc:31:
/hpc/home/kbw29/Code/brian2Genn/Output/brianlib/clocks.h:12:3: warning: extra ‘;’ [-Wpedantic]
  };
   ^
/hpc/home/kbw29/Code/brian2Genn/Output/brianlib/clocks.h:13:2: warning: extra ‘;’ [-Wpedantic]
 };
  ^
In file included from /hpc/home/kbw29/Code/brian2Genn/Output/magicnetwork_model.cpp:14:0,
                 from generator.cc:31:
/hpc

RuntimeError: Project compilation failed (Command ['/hpc/home/kbw29/Code/genn/bin/genn-buildmodel.sh', '-i', '/hpc/home/kbw29/Code/brian2Genn:/hpc/home/kbw29/Code/brian2Genn/Output:/hpc/home/kbw29/Code/brian2Genn/Output/brianlib/randomkit', 'magicnetwork_model.cpp'] failed with error code 50).
See the output above (if any) for more details.

#### Benchmarking

In [ ]:
time.time() - start

In [ ]:
# For benchmarking time usage per step ... do not use with multiple runs 
profiling_summary(show = 10)

# Visualization 

In [ ]:
plot_potential(spikeMonitor_GR, stateMonitor_GR, 2)

In [ ]:
plot_spikes(spikeMonitor_GR, stateMonitor_GR, 2)

#### By trial 

In [ ]:
plot_population(prMonitor_GR, spikeMonitor_GR, 'GranuleCells', 100, 10)

In [ ]:
plot_population_Multi(prMulti_GR_smooth[0], prMulti_GR_t[0], spikeMonitor_GR, 'GranuleCells', 100, 10)

In [ ]:
plot_population_Multi(prMulti_GR_smooth[1], prMulti_GR_t[1], spikeMonitor_GR, 'GranuleCells', 100, 10)

#### Individual Granule Cells 

In [ ]:
tempAppend = []
neuronNumber = 0

for i in range(len(spikeMonitor_Multi_GR)):
    tempAppend.extend(pd.array(spikeMonitor_Multi_GR[i][neuronNumber])*1000)

sns.kdeplot(tempAppend, bw_adjust = 0.5)

In [ ]:
tempAppend = []
neuronNumber = 1

for i in range(len(spikeMonitor_Multi_GR)):
    tempAppend.extend(pd.array(spikeMonitor_Multi_GR[i][neuronNumber])*1000)

sns.kdeplot(tempAppend, bw_adjust = 0.2)

In [ ]:
tempAppend = []
neuronNumber = 2

for i in range(len(spikeMonitor_Multi_GR)):
    tempAppend.extend(pd.array(spikeMonitor_Multi_GR[i][neuronNumber])*1000)

sns.kdeplot(tempAppend, bw_adjust = 0.2)

In [ ]:
tempAppend = []
neuronNumber = 43

for i in range(len(spikeMonitor_Multi_GR)):
    tempAppend.extend(pd.array(spikeMonitor_Multi_GR[i][neuronNumber])*1000)

sns.kdeplot(tempAppend, bw_adjust = 0.2)

#### Population Readouts

##### Granule  Cells

In [ ]:
# Plotting average population firing rate across all runs 
average_prMulti = np.mean(prMulti_GR_smooth, axis = 0)

plot_population_Multi_2(average_prMulti, prMulti_GR_t[1], 'GranuleCells')

In [ ]:
active_percent(spikeMonitor_GR, binSize = 10, durationSimulation = 2, numNeurons = numGC, ymax = 20)

In [ ]:
averaged_ActivePercent(spkActive_Multi_GR, spikeMonitor_GR, binSize = 10, durationSimulation = 2, numNeurons = numGC, ymax = 10)

### Purkinje

In [ ]:
plot_potential(spikeMonitor_PKJ, stateMonitor_PKJ, 2)

In [ ]:
plot_spikes(spikeMonitor_PKJ, stateMonitor_PKJ, 2)

In [ ]:
# Plotting average population firing rate across all runs 
average_prMulti_PKJ = np.mean(prMulti_PKJ_smooth, axis = 0)

plot_population_Multi_2(average_prMulti_PKJ, prMulti_PKJ_t[1], 'Purkinje')

## Basket Cell 

In [ ]:
# Plotting average population firing rate across all runs 
average_prMulti_BS = np.mean(prMulti_BS_smooth, axis = 0)

plot_population_Multi_2(average_prMulti_BS, prMulti_BS_t[1], 'Basket Cells')

## Golgi 

In [ ]:
# Plotting average population firing rate across all runs 
average_prMulti_GO = np.mean(prMulti_GO_smooth, axis = 0)

plot_population_Multi_2(average_prMulti_GO, prMulti_GO_t[1], 'Golgi')